GDD Classify Pipeline

1. Step Set the run configuration.
Be sure to set java memory not over your usable RAM
If Metanome fails, be sure to use JAVA Version 18.0.2.1

In [1]:
from ipywidgets import Textarea, Button, VBox, Label
import json
from notebook_ui import config_editor
# Load your config

config_editor()

Select a file that you want to classify:

In [2]:
from notebook_ui import upload_widget
import json

# Load your config
with open("config.json", "r") as f:
    config = json.load(f)

# Display widget
upload_widget(config)

Run the classification pipeline on the dataset:

In [3]:
%run test_pipeline.py

⚠️ Error processing file with encoding utf-8-sig: [Errno 2] No such file or directory: 'TrainingData/fakeData/anne_car_data_1.csv'
Starting full test pipeline with metrics...
Cleaning CSV files in UserData/realData ...
Cleaning UserData/realData\estat_sbs_sc_ovw_filtered_en.csv ...
Reading and processing: UserData/realData\estat_sbs_sc_ovw_filtered_en.csv


Reading: 100%|██████████| 1/1 [00:00<00:00, 401.64it/s]

⚠️ Only 106 rows found. Sampling skipped.
✅ Cleaned CSV saved to: UserData/realData\estat_sbs_sc_ovw_filtered_en.csv.tmp
Processing is complete and file has been overwritten with sampled data.


[2025-03-29 11:55:53] 🚀 Running Metanome on: UserData/realData\estat_sbs_sc_ovw_filtered_en.csv
[2025-03-29 11:55:53] ✅ Metanome completed in 419 ms
Cleaning CSV files in UserData/fakeData ...
Found 1 datasets in 'UserData/realData'. Starting classification...
UserData/realData\estat_sbs_sc_ovw_filtered_en.csv classified as real in 35 ms.
